# Camera Pan Tilt

## 1. Camare Streaming On

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
import time
import numpy as np

from bcam.bcam import Camera, frame_dp, bgr8_to_jpeg

In [ ]:
width_cap = 800
height_cap = 600
fps_cap = 4

resize = 1

width_dp = width_cap * resize
height_dp = height_cap * resize

In [ ]:
camera = Camera(width=width_cap, height=height_cap, fps=fps_cap, is_usb=False, flip=2)

image_widget = widgets.Image(format='jpeg', width=width_dp, height=height_dp)

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=frame_dp)

In [ ]:
display(image_widget)

## 2. Activate Pan Tilt

In [ ]:
from utils.servoctrl import PanTilt
pt = PanTilt(bus=0)
pt.add_rl_servo(channel=14, max_angle=180, min_angle=0)
pt.add_ud_servo(channel=15, max_angle=180, min_angle=0)

## 3. Add Widgets for Control Pan Tilt

### 1) Buttons Widgets

In [ ]:
import ipywidgets.widgets as widgets
from ipywidgets import Button, HBox, VBox

button_layout = widgets.Layout(width='100px', height='80px', align_self='center')

step_move = ['↖', '↑', '↗', '←', 'reset', '→', '↙', '↓', '↘']
step_items = [Button(description=i, layout=button_layout) for i in step_move]

far_move = ['◤', '▲', '◥', '◄', 'reset', '►', '◣', '▼', '◢']
far_items = [Button(description=i, layout=button_layout) for i in far_move]

#'success', 'info', 'warning', 'danger' or ''
#step_items[4].button_style='info'
#far_items[4].button_style='info'

row1 = HBox([step_items[0], step_items[1], step_items[2]])
row2 = HBox([step_items[3], step_items[4], step_items[5]])
row3 = HBox([step_items[6], step_items[7], step_items[8]])
com_a = VBox([row1, row2, row3])

row4 = HBox([far_items[0], far_items[1], far_items[2]])
row5 = HBox([far_items[3], far_items[4], far_items[5]])
row6 = HBox([far_items[6], far_items[7], far_items[8]])
com_b = VBox([row4, row5, row6])

In [ ]:
display(widgets.HBox([com_a, com_b]))

##### --- Link Buttons Widgets to Actions

In [ ]:
#step: '↖', '↑', '↗', '←', 'reset', '→', '↙', '↓', '↘'
step_items[0].on_click(lambda x: pt.go_up_left())
step_items[1].on_click(lambda x: pt.go_up())
step_items[2].on_click(lambda x: pt.go_up_right())
step_items[3].on_click(lambda x: pt.go_left())
step_items[4].on_click(lambda x: pt.reset())
step_items[5].on_click(lambda x: pt.go_right())
step_items[6].on_click(lambda x: pt.go_down_left())
step_items[7].on_click(lambda x: pt.go_down() )
step_items[8].on_click(lambda x: pt.go_down_right())

#far: '◤', '▲', '◥', '◄', 'reset', '►', '◣', '▼', '◢'
far_items[0].on_click(lambda x: pt.upper_left())
far_items[1].on_click(lambda x: pt.far_up())
far_items[2].on_click(lambda x: pt.upper_right())
far_items[3].on_click(lambda x: pt.far_left())
far_items[4].on_click(lambda x: pt.reset())
far_items[5].on_click(lambda x: pt.far_right())
far_items[6].on_click(lambda x: pt.lower_left())
far_items[7].on_click(lambda x: pt.far_down() )
far_items[8].on_click(lambda x: pt.lower_right())

# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.

### 2) Slider Widgets

In [ ]:
rl_slider = widgets.FloatSlider(min=-1, max=1, value=0, step=0.02, description='rl')

def rl_val(val):
    return int((val * (pt.rl.max_angle - pt.rl.min_angle) + pt.rl.max_angle + pt.rl.min_angle) / 2)

def rl_move1(change):
    val = change['new']
    pt.to_angle(rl=rl_val(val))
    
rl_slider.observe(rl_move1, names=['value'])

In [ ]:
ud_slider = widgets.FloatSlider(min=-1, max=1, value=0, step=0.02, description='ud')

def ud_val(val):
    return int((val * (pt.rl.max_angle - pt.rl.min_angle) + pt.rl.max_angle + pt.rl.min_angle) / 2)

def ud_move1(change):
    val = change['new']
    pt.to_angle(ud=ud_val(val))
    
ud_slider.observe(ud_move1, names=['value'])

In [ ]:
display(rl_slider, ud_slider)

### 3) Text Widgets

In [ ]:
rl_textbox = widgets.IntText(layout=widgets.Layout(width='148px'), value=180, description='rl:')

def rl_move2(change):
    val = change['new']
    pt.to_angle(rl=val)
    
rl_textbox.observe(rl_move2, names=['value'])

In [ ]:
ud_textbox = widgets.IntText(layout=widgets.Layout(width='148px'), value=180, description='ud:')

def ud_move2(change):
    val = change['new']
    pt.to_angle(ud=val)
    
ud_textbox.observe(ud_move2, names=['value'])

In [ ]:
display(HBox([rl_textbox, ud_textbox]))

### 4) Functions: Snapshot, Save Reset, Read Position

In [ ]:
import ipywidgets.widgets as widgets
from ipywidgets import Button, HBox, VBox

button_layout = widgets.Layout(width='100px', height='80px', align_self='center')

func_act = ['snapshot', 'save reset', 'position']
func_items = [Button(description=i, layout=button_layout) for i in func_act]

func_items[0].button_style='warning'
func_items[1].button_style='info'
func_items[2].button_style=''

func_box = VBox([func_items[0], func_items[1], func_items[2]])

#### -- Snapshot

In [ ]:
import os

snap_dir = 'snap'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(snap_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

In [ ]:
snap_count = widgets.IntText(description='count:', layout=widgets.Layout(width='140px'), value=len(os.listdir(snap_dir)))

def save_snap():
    localtime = time.strftime("%Y%m%d_%H%M%S", time.localtime())
    image_path = os.path.join(snap_dir, localtime + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
    snap_count.value = len(os.listdir(snap_dir))
    
func_items[0].on_click(lambda x: save_snap())

In [ ]:
display(func_items[0], snap_count)

#### -- Save Reset

In [ ]:
func_items[1].on_click(lambda x: pt.change_reset())
display(func_items[1])

#### -- Read Position

In [ ]:
text_widget = widgets.Text(layout=widgets.Layout(width='220px'), value="", description='Position:')

def read_pos():
    pos = 'rl: {}   |   ud: {}'.format(pt.read()[0], pt.read()[1])
    text_widget.value = pos
    
func_items[2].on_click(lambda x: read_pos())

In [ ]:
display(func_items[2], text_widget)

## Control Interface

In [ ]:
widgets_set1 = HBox([com_a, func_box, com_b])
widgets_set2 = VBox([text_widget, HBox([rl_textbox, ud_textbox, snap_count])])
widgets_set3 = HBox([VBox([rl_slider, ud_slider]), widgets_set2])

In [ ]:
display(image_widget, widgets_set3, widgets_set1)

## 4. Add PS4 Joystick for Control Pan Tilt

In [ ]:
controller = widgets.Controller(index=0)  # replace with index of your controller
display(controller)

In [ ]:
controller.buttons[12].observe(lambda x: pt.go_up(), names='value') # Up
controller.buttons[13].observe(lambda x: pt.go_down(), names='value') # Down
controller.buttons[14].observe(lambda x: pt.go_left(), names='value') # Left
controller.buttons[15].observe(lambda x: pt.go_right(), names='value') # Right

controller.buttons[3].observe(lambda x: pt.far_up(), names='value') # Far Up
controller.buttons[0].observe(lambda x: pt.far_down(), names='value') # Far Down
controller.buttons[2].observe(lambda x: pt.far_left(), names='value') # Far Left
controller.buttons[1].observe(lambda x: pt.far_right(), names='value') # Far Right

controller.buttons[8].observe(lambda x: pt.reset(), names='value')
controller.buttons[9].observe(lambda x: pt.change_reset(), names='value')

controller.buttons[5].observe(lambda x: save_snap(), names='value')

controller.axes[0].observe(rl_move1, names=['value'])
controller.axes[3].observe(ud_move1, names=['value'])